In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
import json
import shap
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

print("All libraries imported successfully")
print("SHAP version:", shap.__version__)

# Load preprocessed data
X_train = np.load('../regression_processed_data/X_train.npy')
X_test = np.load('../regression_processed_data/X_test.npy')
y_train = np.load('../regression_processed_data/y_train.npy')
y_test = np.load('../regression_processed_data/y_test.npy')
feature_names = json.load(open('../regression_processed_data/feature_names.json'))

print(f"Data Loaded:")
print(f"   Training samples: {X_train.shape[0]}")
print(f"   Test samples: {X_test.shape[0]}")
print(f"   Features: {X_train.shape[1]}")
print(f"   Feature names: {len(feature_names)}")

# Dynamically load all available base models
models = {}
model_dir = '../saved_base_models'

# Check if directory exists
if not os.path.exists(model_dir):
    print(f"Error: Directory {model_dir} does not exist")
    sys.exit(1)

# Get all .pkl files in the directory
available_models = [f.replace('_model.pkl', '') for f in os.listdir(model_dir) 
                   if f.endswith('_model.pkl')]

if not available_models:
    print(f"Error: No base models found in {model_dir}")
    sys.exit(1)

print(f"\nFound {len(available_models)} base models:")
for model_name in available_models:
    print(f"   - {model_name}")

# Load all available models
for model_name in available_models:
    model_path = f'{model_dir}/{model_name}_model.pkl'
    try:
        models[model_name] = joblib.load(model_path)
        print(f"Loaded: {model_name.upper()}")
    except Exception as e:
        print(f"Error loading {model_name}: {str(e)}")

if not models:
    print("Error: No models were successfully loaded")
    sys.exit(1)

print("\n" + "=" * 70)
print("SHAP (SHAPLEY VALUES) FEATURE IMPORTANCE ANALYSIS - BASE MODELS")
print("=" * 70)

shap_results = {}

print("\nCalculating SHAP values for all base models...")
print("This may take several minutes...")

for name, model in models.items():
    print(f"\n[{name.upper()}] Computing SHAP values...")
    
    try:
        # Determine model type and use appropriate explainer
        if 'random_forest' in name.lower() or 'decision_tree' in name.lower():
            # Tree-based models use TreeExplainer
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X_test)
        
        elif 'neural' in name.lower() or 'mlp' in name.lower():
            # Neural network models use KernelExplainer with smaller sample
            background = shap.sample(X_train, 100)
            explainer = shap.KernelExplainer(model.predict, background)
            shap_values = explainer.shap_values(X_test[:100])
        
        else:
            # Other models (KNN, SVR, Ridge, etc.) use KernelExplainer
            background = shap.sample(X_train, 100)
            explainer = shap.KernelExplainer(model.predict, background)
            shap_values = explainer.shap_values(X_test[:100])
        
        shap_results[name] = {
            'explainer': explainer,
            'shap_values': shap_values,
            'feature_importance': np.abs(shap_values).mean(axis=0)
        }
        
        print(f"   SHAP computation complete for {name.upper()}")
    
    except Exception as e:
        print(f"   Error computing SHAP for {name}: {str(e)}")
        continue

if not shap_results:
    print("Error: No SHAP values were computed successfully")
    sys.exit(1)

print("\n" + "=" * 70)
print("SHAP FEATURE IMPORTANCE RANKINGS")
print("=" * 70)

shap_importance_df_list = []

for name, shap_data in shap_results.items():
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'SHAP_Importance': shap_data['feature_importance']
    }).sort_values('SHAP_Importance', ascending=False)
    
    shap_importance_df_list.append(importance_df)
    
    print(f"\n{name.upper()} - Top 10 Features by SHAP:")
    print(f"{'Rank':<6}{'Feature':<25}{'SHAP Importance':<20}")
    print("-" * 51)
    
    for i, (idx, row) in enumerate(importance_df.head(10).iterrows(), 1):
        print(f"{i:<6}{row['Feature']:<25}{row['SHAP_Importance']:<20.6f}")

# Combine SHAP importance across all models
combined_shap = pd.DataFrame({
    'Feature': feature_names
})

for name, shap_data in shap_results.items():
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        f'{name}_shap': shap_data['feature_importance']
    })
    combined_shap = combined_shap.merge(importance_df, on='Feature')

shap_cols = [col for col in combined_shap.columns if col.endswith('_shap')]
combined_shap['avg_shap'] = combined_shap[shap_cols].mean(axis=1)
combined_shap = combined_shap.sort_values('avg_shap', ascending=False)

print("\n" + "=" * 70)
print("CONSENSUS SHAP FEATURE IMPORTANCE (Averaged Across All Base Models)")
print("=" * 70)

print(f"\n{'Rank':<6}{'Feature':<25}{'Average SHAP':<20}")
print("-" * 51)

for i, (idx, row) in enumerate(combined_shap.head(12).iterrows(), 1):
    print(f"{i:<6}{row['Feature']:<25}{row['avg_shap']:<20.6f}")

print("\n" + "=" * 70)
print("SHAP VISUALIZATIONS")
print("=" * 70)

# Create output directory
os.makedirs('../saved_base_models/shap_analysis', exist_ok=True)

# Calculate grid size for subplots
num_models = len(shap_results)
num_cols = 3
num_rows = (num_models + 1 + num_cols - 1) // num_cols

fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 6 * num_rows))
axes = axes.flatten()

colors = ['#2ecc71', '#3498db', '#e74c3c', '#9b59b6', '#f39c12', '#1abc9c', '#e67e22', '#34495e']

for idx, (name, shap_data) in enumerate(shap_results.items()):
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'SHAP_Importance': shap_data['feature_importance']
    }).sort_values('SHAP_Importance', ascending=False).head(10)
    
    axes[idx].barh(importance_df['Feature'], importance_df['SHAP_Importance'],
                   color=colors[idx % len(colors)], edgecolor='black', alpha=0.7)
    axes[idx].set_xlabel('Mean |SHAP Value|', fontweight='bold')
    axes[idx].set_title(f'{name.upper()} - SHAP Feature Importance', fontweight='bold')
    axes[idx].invert_yaxis()
    axes[idx].grid(axis='x', alpha=0.3)

# Add consensus plot
axes[num_models].barh(combined_shap.head(12)['Feature'], combined_shap.head(12)['avg_shap'],
             color='darkgreen', edgecolor='black', alpha=0.7)
axes[num_models].set_xlabel('Average SHAP Importance', fontweight='bold')
axes[num_models].set_title('CONSENSUS - Average SHAP Across All Base Models', fontweight='bold')
axes[num_models].invert_yaxis()
axes[num_models].grid(axis='x', alpha=0.3)

# Hide unused subplots
for idx in range(num_models + 1, len(axes)):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.savefig('../saved_base_models/shap_analysis/base_models_shap_comparison.png', dpi=300, bbox_inches='tight')
print("\nSaved: ../saved_base_models/shap_analysis/base_models_shap_comparison.png")
plt.show()

# Find the best tree-based model for detailed analysis
tree_based_models = {k: v for k, v in shap_results.items() 
                     if 'random_forest' in k.lower() or 'decision_tree' in k.lower()}

if tree_based_models:
    # Select random forest if available, otherwise use decision tree
    best_model_name = next((k for k in tree_based_models.keys() if 'random_forest' in k.lower()), 
                           next(iter(tree_based_models.keys())))
    
    print("\n" + "=" * 70)
    print(f"DETAILED SHAP PLOTS FOR {best_model_name.upper()}")
    print("=" * 70)
    
    best_shap_values = shap_results[best_model_name]['shap_values']
    best_explainer = shap_results[best_model_name]['explainer']
    
    sample_size = min(100, len(best_shap_values))
    
    # Summary plot
    print(f"\n1. SHAP Summary Plot (Feature Impact Distribution)")
    plt.figure(figsize=(12, 8))
    shap.summary_plot(best_shap_values[:sample_size], 
                     X_test[:sample_size], 
                     feature_names=feature_names,
                     show=False)
    plt.title(f'SHAP Summary Plot - {best_model_name.upper()}', fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig(f'../saved_base_models/shap_analysis/{best_model_name}_summary_plot.png', dpi=300, bbox_inches='tight')
    print(f"Saved: ../saved_base_models/shap_analysis/{best_model_name}_summary_plot.png")
    plt.show()
    
    # Bar plot
    print(f"\n2. SHAP Summary Plot (Bar)")
    plt.figure(figsize=(12, 8))
    shap.summary_plot(best_shap_values[:sample_size], 
                     X_test[:sample_size], 
                     feature_names=feature_names,
                     plot_type="bar",
                     show=False)
    plt.title(f'SHAP Feature Importance - {best_model_name.upper()}', fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig(f'../saved_base_models/shap_analysis/{best_model_name}_bar_plot.png', dpi=300, bbox_inches='tight')
    print(f"Saved: ../saved_base_models/shap_analysis/{best_model_name}_bar_plot.png")
    plt.show()
    
    # Waterfall plot
    print(f"\n3. SHAP Waterfall Plot (First Prediction)")
    plt.figure(figsize=(12, 8))
    shap.waterfall_plot(
        shap.Explanation(values=best_shap_values[0],
                        base_values=best_explainer.expected_value,
                        data=X_test[0],
                        feature_names=feature_names),
        show=False
    )
    plt.title(f'SHAP Waterfall Plot - Sample Prediction ({best_model_name.upper()})', 
             fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig(f'../saved_base_models/shap_analysis/{best_model_name}_waterfall_plot.png', dpi=300, bbox_inches='tight')
    print(f"Saved: ../saved_base_models/shap_analysis/{best_model_name}_waterfall_plot.png")
    plt.show()
    
    # Dependence plots
    top_features = combined_shap.head(3)['Feature'].tolist()
    
    print(f"\n4. SHAP Dependence Plots (Top 3 Features)")
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    for idx, feature in enumerate(top_features):
        feature_idx = feature_names.index(feature)
        
        shap.dependence_plot(
            feature_idx,
            best_shap_values[:sample_size],
            X_test[:sample_size],
            feature_names=feature_names,
            ax=axes[idx],
            show=False
        )
        axes[idx].set_title(f'SHAP Dependence: {feature}', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(f'../saved_base_models/shap_analysis/{best_model_name}_dependence_plots.png', dpi=300, bbox_inches='tight')
    print(f"Saved: ../saved_base_models/shap_analysis/{best_model_name}_dependence_plots.png")
    plt.show()
    
    # Force plots
    print("\n" + "=" * 70)
    print("SHAP FORCE PLOTS FOR INDIVIDUAL PREDICTIONS")
    print("=" * 70)
    
    sample_indices = [0, 1, 2]
    
    for sample_idx in sample_indices:
        print(f"\nForce plot for sample {sample_idx}")
        plt.figure(figsize=(20, 3))
        
        shap.force_plot(
            best_explainer.expected_value,
            best_shap_values[sample_idx],
            X_test[sample_idx],
            feature_names=feature_names,
            matplotlib=True,
            show=False
        )
        
        plt.tight_layout()
        plt.savefig(f'../saved_base_models/shap_analysis/force_plot_sample_{sample_idx}.png', 
                    dpi=300, bbox_inches='tight')
        print(f"Saved: ../saved_base_models/shap_analysis/force_plot_sample_{sample_idx}.png")
        plt.show()

# Save SHAP importance data
for name, shap_data in shap_results.items():
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'SHAP_Importance': shap_data['feature_importance']
    }).sort_values('SHAP_Importance', ascending=False)
    
    importance_df.to_csv(f'../saved_base_models/shap_analysis/{name}_shap_importance.csv', index=False)

combined_shap.to_csv('../saved_base_models/shap_analysis/combined_shap_importance.csv', index=False)

print("\n" + "=" * 70)
print("SHAP ANALYSIS COMPLETE")
print("=" * 70)
print(f"\nSHAP importance data saved to: ../saved_base_models/shap_analysis/")
print(f"\nFiles created:")
for name in shap_results.keys():
    print(f"   {name}_shap_importance.csv")
print(f"   combined_shap_importance.csv")
print(f"   base_models_shap_comparison.png")
if tree_based_models:
    print(f"   {best_model_name}_summary_plot.png")
    print(f"   {best_model_name}_bar_plot.png")
    print(f"   {best_model_name}_waterfall_plot.png")
    print(f"   {best_model_name}_dependence_plots.png")
    print(f"   force_plot_sample_0.png, force_plot_sample_1.png, force_plot_sample_2.png")

c:\Users\rosem\Documents\AJ\Thesis-Implementation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All libraries imported successfully
SHAP version: 0.49.1
Data Loaded:
   Training samples: 888
   Test samples: 222
   Features: 16
   Feature names: 16
Loaded: KNN
Loaded: DECISION_TREE
Loaded: RANDOM_FOREST
Loaded: SVR
Loaded: RIDGE

SHAP (SHAPLEY VALUES) FEATURE IMPORTANCE ANALYSIS - BASE MODELS

Calculating SHAP values for all base models...
This may take several minutes...

[KNN] Computing SHAP values...


  4%|▍         | 4/100 [00:02<01:06,  1.44it/s]


KeyboardInterrupt: 